In [9]:
import pyAgrum as gum

In [10]:
# Crear la red bayesiana
bn = gum.BayesNet('SecuritySystem')

In [11]:
# Añadir nodos
http_requests = bn.add(gum.LabelizedVariable('HTTP_Requests', 'HTTP Requests', 2))
attacks = bn.add(gum.LabelizedVariable('Attacks', 'Attacks', 2))
serious_failures = bn.add(gum.LabelizedVariable('Serious_Failures', 'Serious Failures', 2))
firewall = bn.add(gum.LabelizedVariable('Firewall', 'Firewall', 2))
recovery_system = bn.add(gum.LabelizedVariable('Recovery_System', 'Recovery System', 2))

In [12]:
# Añadir arcos
bn.addArc(http_requests, attacks)
bn.addArc(attacks, serious_failures)
bn.addArc(firewall, attacks)
bn.addArc(recovery_system, serious_failures)

In [13]:
# Definir las tablas de probabilidad condicional (CPTs)
bn.cpt(http_requests).fillWith([0.1, 0.9])  # 10% ataques, 90% no ataques
bn.cpt(firewall).fillWith([0.95, 0.05])  # 95% activo, 5% no activo
bn.cpt(recovery_system).fillWith([0.75, 0.25])  # 75% activo, 25% no activo
bn.cpt(attacks)[{'Firewall': 0, 'HTTP_Requests': 0}] = [0.25, 0.75]  # 25% no ataques si no hay peticiones y firewall inactivo, 75% ataques
bn.cpt(attacks)[{'Firewall': 0, 'HTTP_Requests': 1}] = [0, 1]  # 0% no ataques si hay peticiones y firewall inactivo, 100% ataques
bn.cpt(attacks)[{'Firewall': 1, 'HTTP_Requests': 0}] = [1, 0]  # 100% no ataques si no hay peticiones y firewall activo, 0% ataques
bn.cpt(attacks)[{'Firewall': 1, 'HTTP_Requests': 1}] = [0, 1]  # 0% no ataques si hay peticiones y firewall activo, 100% ataques
bn.cpt(serious_failures)[{'Recovery_System': 0, 'Attacks': 0}] = [0.25, 0.75]  # 25% fallo grave si no hay recuperación y no hay ataques, 75% no fallo
bn.cpt(serious_failures)[{'Recovery_System': 0, 'Attacks': 1}] = [0, 1]  # 100% fallo grave si no hay recuperación y hay ataques
bn.cpt(serious_failures)[{'Recovery_System': 1, 'Attacks': 0}] = [0.5, 0.5]  # 50% fallo grave si hay recuperación y no hay ataques, 50% no fallo
bn.cpt(serious_failures)[{'Recovery_System': 1, 'Attacks': 1}] = [0, 1]  # 100% fallo grave si hay recuperación y hay ataques

In [14]:
# Crear un objeto de inferencia
ie = gum.LazyPropagation(bn)

In [17]:
# Calcular la probabilidad de un fallo grave en cualquier momento
ie.makeInference()
prob_serious_failure = ie.posterior(serious_failures)[0]
print(f"Probabilidad de un fallo grave en cualquier momento: {prob_serious_failure*100:.2f}%")

Probabilidad de un fallo grave en cualquier momento: 0.90%


In [21]:
# Calcular la probabilidad de un fallo grave sin cortafuegos
ie.setEvidence({'Firewall': 1})
ie.makeInference()
prob_serious_failure_no_firewall = ie.posterior(serious_failures)[0]
print(f"Probabilidad de un fallo grave sin cortafuegos: {prob_serious_failure_no_firewall*100:.2f}%")

Probabilidad de un fallo grave sin cortafuegos: 3.12%


In [22]:
# Función para obtener los nombres de las variables de una red bayesiana
def get_variable_names(bn):
    return [bn.variable(i).name() for i in bn.nodes()]

In [29]:
# Función para verificar la independencia marginal entre pares de variables
def check_marginal_independence(bn):
    independence_results = {}
    for var1 in bn.nodes():
        for var2 in bn.nodes():
            if var1 != var2:
                independence_results[(bn.variable(var1).name(), bn.variable(var2).name())] = bn.isIndependent(var1, var2)
    return independence_results

In [24]:
# Obtener nombres de las variables
variable_names = get_variable_names(bn)
print(f"Nombres de las variables: {variable_names}")

Nombres de las variables: ['HTTP_Requests', 'Attacks', 'Serious_Failures', 'Firewall', 'Recovery_System']


In [25]:
# Verificar independencia marginal entre pares de variables
independence_results = check_marginal_independence(bn)
for vars_pair, is_independent in independence_results.items():
    print(f"{vars_pair}: {'Independientes' if is_independent else 'Dependientes'}")

('HTTP_Requests', 'Attacks'): Dependientes
('HTTP_Requests', 'Serious_Failures'): Dependientes
('HTTP_Requests', 'Firewall'): Independientes
('HTTP_Requests', 'Recovery_System'): Independientes
('Attacks', 'HTTP_Requests'): Dependientes
('Attacks', 'Serious_Failures'): Dependientes
('Attacks', 'Firewall'): Dependientes
('Attacks', 'Recovery_System'): Independientes
('Serious_Failures', 'HTTP_Requests'): Dependientes
('Serious_Failures', 'Attacks'): Dependientes
('Serious_Failures', 'Firewall'): Dependientes
('Serious_Failures', 'Recovery_System'): Dependientes
('Firewall', 'HTTP_Requests'): Independientes
('Firewall', 'Attacks'): Dependientes
('Firewall', 'Serious_Failures'): Dependientes
('Firewall', 'Recovery_System'): Independientes
('Recovery_System', 'HTTP_Requests'): Independientes
('Recovery_System', 'Attacks'): Independientes
('Recovery_System', 'Serious_Failures'): Dependientes
('Recovery_System', 'Firewall'): Independientes
